<a href="https://colab.research.google.com/github/katrin2202/text-summarization-in-nlp/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
sample_text = """13 февраля 120 лет назад родился один из основателей отечественной ядерной энергетики Анатолий Александров. Полное перечисление его чинов, званий и наград займет не одну строчку — физик, академик, трижды Герой Социалистического Труда, лауреат Ленинской, Государственной и четырех Сталинских премий, президент Академии наук СССР в 1975–1986 годах и т.д. Трудно даже представить, сколь колоссальный масштаб работ выполнил этот человек за одну жизнь.


© ТАСС
Первый интерес
Видео дня

Будущий знаменитый ученый появился на свет в городке Тараща Киевской губернии. Он был третьим ребенком в семье, в три года потерял мать, после чего опеку взяла бабушка. Получив довольно хорошее домашнее образование, Анатолий Александров примкнул к физико-химическому кружку, где вскоре стал инициатором создания электротехнической группы. Гражданская война застала юношу, когда он обучался в Киевском реальном училище, — поддавшись агитации, парень записался в русские добровольческие части для защиты города от петлюровцев и большевиков.

В 16-летнем возрасте Александров стал юнкером и его зачислили в армию барона Врангеля пулеметчиком; за проявленную в боях доблесть был удостоен трех Георгиевских крестов. Когда разгромленные белогвардейцы под натиском Красной армии оставили Крым, он решил не покидать малую родину, а остаться (закопав награды). Попал в плен. Об этом периоде своей жизни Анатолий Александров почти никогда не рассказывал.

Когда наступил мир и жизнь в стране стала налаживаться, молодой Александров, будучи учителем физики одной из киевских школ, увлекся изучением свойств диэлектриков (вещества и материалы, которые относительно плохо проводят электрический ток). Основные работы в этой области, весьма модной тогда, велись в Ленинградском физико-техническом институте (ЛФТИ) под руководством академика Абрама Иоффе. Маститый ученый с мировым именем узнал о проводимых в Киеве исследованиях и направил туда нескольких ведущих специалистов, чтобы наладить контакты. Так в начале 1930 года состоялась первая встреча Анатолия Александрова и Игоря Курчатова — последнего уже хорошо знали в научном мире после публикации ряда важных работ. Два отечественных гения сразу прониклись симпатией друг к другу, которая переросла в крепкую дружбу.

Примерно через полгода Александрова и некоторых его сподвижников перевели в Ленинград в ЛФТИ, где под началом академика Иоффе он проработал до 1946 года, защитил кандидатскую и докторские диссертации.

«Жизнь наша в бытовом плане была тяжелой. Впервые мы жили вне семьи, спали в общежитии, в бывшем кабинете великого князя, где стояло восемь коек и нужно было укрываться одеялом с головой, чтобы мыши не объели тебе уши. Вместо хорошей домашней пищи приходилось сидеть полуголодным и есть всякую дрянь… Но это был Физико-технический институт! В нем к нам относились отлично, всячески помогая, каждый день, проведенный там, приносил массу интересного, расширял кругозор, мы росли как на дрожжах»

Из книги "А.П. Александров: документы и воспоминания. К 100-летию со дня рождения"

Размагничивание кораблей в Великую Отечественную войну
Работой, имевшей важное оборонное значение, стали эксперименты по борьбе с неконтактными магнитными минами — смертоносным оружием, которое преимущественно сбрасывалось с самолетов-миноносцев в фарватерах, бухтах, вблизи портов. Если Курчатов, видимо, способствовал переезду Александрова в Ленинград и подключению его к научным исследованиям ЛФТИ, то теперь последний предложил включить в группу по размагничиванию кораблей Игоря Курчатова — оба убыли в Севастополь в начале августа 1941 года.

Группа физиков использовала так называемое обмоточное размагничивание корабля с помощью мощных магнитных полей "по методу ЛФТИ". Порядка 100 кораблей Черноморского флота подверглись размагничиванию, что заметно способствовало резкому сокращению наших потерь от минного оружия. Немцы, конечно, модернизировали оружие: ставили на магнитные мины световые и гидравлические датчики. Но все же эффект от проведенной работы был значительным. Это относилось и к акватории Нижней Волги, где магнитные мины также создавали большую угрозу судоходству.

По признанию самого Анатолия Александрова, он особо гордился двумя медалями — "За оборону Севастополя" и "За оборону Сталинграда", полученными как раз за проведение работ по размагничиванию кораблей и судов.

"Урановая проблема" и ее решение
Как теперь известно, в сентябре 1942 года Государственный комитет обороны (ГКО) издал распоряжение о необходимости возобновить в охваченной войной стране исследования по урану и представить за полгода доклад о возможности создания у нас урановой бомбы. 12 апреля следующего года, выполняя решение ГКО, Академия наук СССР секретным постановлением организовала лабораторию №2 по исследованию атомного ядра, далее переименованную в Лабораторию измерительных приборов АН СССР (ЛИПАН). Научным руководителем назначили Игоря Курчатова и ознакомили его с разведывательными материалами, полученными из Великобритании.

Понимая, сколь сложную практическую задачу предстоит решить, Курчатов стал расширять круг соратников и привлекать к работе специалистов в области атомной физики и техники, в том числе и Анатолия Александрова (на тот момент он уже был избранным членом-корреспондентом Академии наук СССР). В 1946 году Александрова назначают директором Института физических проблем АН СССР, а в 1955-м его переводят по настоянию Курчатова его заместителем в ЛИПАН, преобразованный через год с небольшим в Институт атомной энергии Академии наук (ИАЭ АН) СССР.

Ученые и организаторы производства в атомной отрасли, с которыми мне посчастливилось работать в разные годы и которые трудились под началом академика Александрова — Николай Кухаркин, Борис Папковский, Юрий Сивинцев, Виктор Сидоренко, Николай Хлопкин, — неизменно отмечали его лидерские качества, умение видеть перспективу работы на 10–15 лет вперед, способность нацелить своих коллег на решение важных и сложных народно-хозяйственных задач. При этом он "никогда не давил авторитетом", а позволял перед принятием решения "всем высказаться, после чего как бы подводил итог обсуждению".

Созидательная функция атома
Наши ученые приложили немало сил, чтобы атомная энергия из средства разрушения превратилась в мощную созидательную силу, которую можно направлять на мирные цели в интересах всего человечества; они также стремились организовать широкое международное сотрудничество в этой области. Неслучайно именно в нашей стране в 100 км от Москвы была построена первая в мире атомная электростанция (АЭС) — Обнинская (26 июня 1954 года состоялся энергетический пуск станции с выдачей электроэнергии в систему Мосэнерго).

Огромная заслуга Александрова состояла в стремлении расширить сферу использования ядерных источников энергии: атомные станции теплоснабжения, ядерные энергоустановки для химической и металлургической промышленности, создание новых материалов, ядерная медицина, работа в космическом пространстве. Он на протяжении почти трех десятилетий оставался научным руководителей самых важных научно-технических программ.

Анатолий Александров непосредственно участвовал в создании "сердца" атомного ледокола "Ленин" и его последователей, ставших истинными покорителями Арктики, а также в установке ВМ-А для первой атомной подводной лодки (АПЛ) К-3 проекта 627 (впоследствии названа "Ленинский комсомол").

При вводе "Ленинского комсомола" в состав Северного флота, согласно традициям, в торжественной обстановке был снят советский флаг и поднят стяг ВМФ. Научный руководитель проекта Александров, приняв от директора завода Евгения Егорова снятое полотнище, поцеловал серп и молот.

Во главе Академии наук
В середине 1970-х годов обстановка в атомной отрасли существенно изменилась. Если за 30 лет до этого все работы в стране шли в обстановке строгой секретности (что было вызвано необходимостью скорейшего создания собственного оружия, исключением возможности шантажа со стороны США), то благодаря широкому международному сотрудничеству, участию в разных научных конференциях имя Александрова и его основные работы были широко известны во всем мире.

Современники Анатолиуса (так его часто называли за глаза) отмечали, что, имея огромный авторитет среди ученых, он был искренне доброжелателен к людям независимо от чинов и званий, а над друзьями любил подшутить и никогда не обижался на розыгрыши в свой адрес. Этот уникальный набор качеств помогал ему успешно сочетать огромную работу по руководству ИАЭ АН (в настоящее время Национальный исследовательский центр "Курчатовский институт") и с крупнейшими государственными научно-техническими программами с плодотворной деятельностью в Академии наук СССР. Поэтому избрание его президентом столь уважаемого органа в 1975 году выглядело вполне естественным. Верный лучшим традициям отечественной науки, на посту президента АН СССР он старался делать все возможное для продвижения прогрессивных научных направлений, развития науки в самых удаленных регионах огромной страны.

Личная ответственность
Анатолий Александров приложил большие усилия, стремясь доказать руководству страны целесообразность масштабного строительства реакторов, преимущественно бескорпусных — типа РБМК (реактор большой мощности канальный): в отличие от ВВЭР (водо-водяной энергетический реактор) они были дешевле в постройке, но не имели защитного кожуха.

По словам знакомых ученого из личной жизни и рабочего окружения, трагические события на Чернобыльской АЭС 26 апреля 1986 года тяжело сказались на его психологическом состоянии, невероятно сильно переживал, принял участие в ликвидации аварии. Да, реактор оказался не столь надежен и безопасен, как ожидалось. Но, как позднее заключила правительственная комиссия СССР, основная проблема все же была в низкой профессиональной подготовке персонала, проведении с нарушениями чрезвычайно опасных экспериментов в ходе плановой остановки одного из энергоблоков.

Анатолий Александров любил проводить свободное время в кругу родных людей, с женой, сыновьями и дочерью, увлекался театром, следил за новинками кино, много читал. В молодые годы предпочитал в отпуске байдарочные походы и практически до конца жизни старался хотя бы пару недель в год прожить в компании друзей в палатке, разбив лагерь на каком-либо острове в низовьях Волги. Этот человек отдыхал с удовольствием, любил и умел работать.

Когда Александрова не стало в 1994 году, его пожелание исполнили: в гроб положили флаг, подаренный ему более 35 лет назад. Выполнили и другую волю выдающегося ученого — похоронили на Митинском кладбище Москвы, там, где упокоились многие ликвидаторы чернобыльской аварии.
"""

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.0/93.0 KB 8.9 MB/s eta 0:00:

In [3]:
from transformers import pipeline

In [22]:
pipe_t5 = pipeline('summarization', model = 'UrukHan/t5-russian-summarization' )

pipe_out = pipe_t5(sample_text)

In [23]:
pipe_out

[{'summary_text': '«Урановая бомба» Анатолия Курчатова: история атомной отрасли, которую мы не можем понять, но все равно будем бороться с ней'}]